In [7]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
from matplotlib import pyplot as plt
import os
import csv

In [3]:
year = 1956

In [14]:
file_path = "../raw data/" + str(year) + "/wedstrijd_ids_" + str(year) + ".csv"


wedstrijd_ids = []
# Save the list to a CSV file
with open(file_path, mode='r', newline='') as file:
    reader = csv.reader(file)
    for row in reader:
        wedstrijd_ids.append(int(row[0]))


In [99]:
wedstrijd_soups = []

for i in range(1):
    wedstrijd_URL = "https://www.voetbalstats.nl/opstelere.php?wid=" + str(wedstrijd_ids[i])

    page = requests.get(wedstrijd_URL)
    soup = BeautifulSoup(page.content, "html.parser")

    wedstrijd_soups.append(soup)
    page_title = soup.title.string if soup.title else "No title found"

    thuis = page_title.split("-")[0].strip()
    uit = page_title.split("-")[1].split("(")[0].strip()
    date = page_title.split()[4][1::]

    file_name = str(wedstrijd_ids[i]) + "," + thuis + "-" + uit + "," + date + ".html"
    file_path = "../raw data/" + str(year) + "/" + file_name

    # Save minified HTML
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(soup.encode(formatter="minimal").decode())

In [86]:
file_name

'138,SC Enschede-MVV,02-06-1957.html'

In [36]:
file_name.split(",")

['138', 'SC Enschede-MVV', '02-06-1957']

In [101]:
# Read the HTML file
with open(file_path, "r", encoding="utf-8") as file:
    loaded_html = file.read()

# Remove unnecessary newlines and extra spaces
cleaned_html = ''.join(loaded_html.splitlines())  # Removes all newline characters

# Parse cleaned HTML
soup = BeautifulSoup(cleaned_html, "html.parser")

In [102]:
td_elements2 = soup.find_all("td", class_="linkere")
td_elements3 = soup.find_all('td', {'align': 'center', 'width': '10%', 'class': 'nl3-wizw'})

scores = []
for td in td_elements3:
    score = td.text.strip().replace(' ', '')  # Removes spaces and trims leading/trailing whitespace
    scores.append(score)

scores_list = []

for i in range(1, len(scores)):
    if scores[i].split("\r")[0][0] > scores[i-1].split("\r")[0][0]:
        team = thuis
    else:
        team = uit
    scores_list.append([scores[i].split("\r")[0], team])


a_tags = soup.find_all("a")

spelers = []

for a_tag in a_tags:
    bal_tag = a_tag.find('img', src='./afbeeldingen/bal.jpg')
    balpen_tag = a_tag.find('img', src='./afbeeldingen/balpen.jpg')
    baled_tag = a_tag.find('img', src='./afbeeldingen/baled.jpg')
    if bal_tag:
        spelers.append(["doelpunt", a_tag.contents[1]])
    if balpen_tag:
        spelers.append(["penalty",a_tag.contents[2]])
    if baled_tag:
        spelers.append(["eigen_doelpunt",a_tag.contents[2]])

print(spelers)

for i in range(len(spelers)):
    speler = spelers[i]
    soort = speler[0]
    #minuut = get_minuut(speler[1].strip().split(". ")[0])
    naam = speler[1].strip().split(". ")[1]
    score = scores_list[i]
    
    print([wedstrijd_ids[i], thuis, uit, soort, naam, score[0], score[1]])

[['doelpunt', '    20. Frans  Rutten '], ['doelpunt', '    28. Gerrit  Moddejonge jr. '], ['doelpunt', '    35. Abe  Lenstra '], ['doelpunt', '    44. Gerrit  Moddejonge jr. '], ['doelpunt', '    50. Henny  Oonk '], ['doelpunt', '    70. Gerrit  Moddejonge jr. '], ['doelpunt', '    80. Frans  Rutten '], ['doelpunt', '    83. Abe  Lenstra '], ['doelpunt', '    88. Gerrit  Moddejonge jr. ']]
[138, 'SC Enschede', 'MVV', 'doelpunt', 'Frans  Rutten', '0-1\t\t\t20.FransRutten28.GerritModdejongejr.1-1\t\t\t35.AbeLenstra2-1\t\t\t44.GerritModdejongejr.3-1\t\t\t50.HennyOonk4-1\t\t\t70.GerritModdejongejr.5-1\t\t\t5-2\t\t\t80.FransRutten83.AbeLenstra6-2\t\t\t88.GerritModdejongejr.7-2', 'MVV']
[251, 'SC Enschede', 'MVV', 'doelpunt', 'Gerrit  Moddejonge jr.', '1-1\t\t\t35.AbeLenstra2-1\t\t\t44.GerritModdejongejr.3-1\t\t\t50.HennyOonk4-1\t\t\t70.GerritModdejongejr.5-1\t\t\t5-2\t\t\t80.FransRutten83.AbeLenstra6-2\t\t\t88.GerritModdejongejr.7-2', 'SC Enschede']
[281, 'SC Enschede', 'MVV', 'doelpunt', '

In [103]:
scores

['7-2\t\t\tMVV',
 '0-1\t\t\t20.FransRutten28.GerritModdejongejr.1-1\t\t\t35.AbeLenstra2-1\t\t\t44.GerritModdejongejr.3-1\t\t\t50.HennyOonk4-1\t\t\t70.GerritModdejongejr.5-1\t\t\t5-2\t\t\t80.FransRutten83.AbeLenstra6-2\t\t\t88.GerritModdejongejr.7-2',
 '1-1\t\t\t35.AbeLenstra2-1\t\t\t44.GerritModdejongejr.3-1\t\t\t50.HennyOonk4-1\t\t\t70.GerritModdejongejr.5-1\t\t\t5-2\t\t\t80.FransRutten83.AbeLenstra6-2\t\t\t88.GerritModdejongejr.7-2',
 '2-1\t\t\t44.GerritModdejongejr.3-1\t\t\t50.HennyOonk4-1\t\t\t70.GerritModdejongejr.5-1\t\t\t5-2\t\t\t80.FransRutten83.AbeLenstra6-2\t\t\t88.GerritModdejongejr.7-2',
 '3-1\t\t\t50.HennyOonk4-1\t\t\t70.GerritModdejongejr.5-1\t\t\t5-2\t\t\t80.FransRutten83.AbeLenstra6-2\t\t\t88.GerritModdejongejr.7-2',
 '4-1\t\t\t70.GerritModdejongejr.5-1\t\t\t5-2\t\t\t80.FransRutten83.AbeLenstra6-2\t\t\t88.GerritModdejongejr.7-2',
 '5-1\t\t\t5-2\t\t\t80.FransRutten83.AbeLenstra6-2\t\t\t88.GerritModdejongejr.7-2',
 '5-2\t\t\t80.FransRutten83.AbeLenstra6-2\t\t\t88.GerritMo

In [92]:
soup


<html>
<head>
<title>
   SC Enschede - MVV (02-06-1957 - Voetbalstats.nl)
  </title>
<meta content="Website: voetbalstats.nl - Rob Regeer" name="author"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="Mozilla/3.0Gold (Win95; I) [Netscape]" name="GENERATOR"/>
<meta content="Zondag 02 juni 1957 - Eredivisie wedstrijd SC Enschede - MVV 7 - 2, Stadion: Stadion Het Diekman,  coach SC Enschede: František  Fadrhonc, coach MVV: Rainer  Hammers,
 scheidsrechter: G.  Florin en overige statistieken. " name="DESCRIPTION"/>
<meta content="voetbal, opstellingen, competitie, gescoord, aanvoerder, keeper, geel, rood, kaarten, toeschouwers, aanvaller, verdediger, middenvelder, seizoen,  1956/1957    " name="KEYWORDS">
<meta content="INDEX, FOLLOW" name="ROBOTS"/>
<meta content="1 MONTH" name="REVISIT-AFTER"/>
<meta content="SC Enschede - MVV (02-06-1957 - Voetbalstats.nl)" property="og:title"/>
<meta content="voetbal" property="og:type"/>
<meta content="" property="og:url"/>
<meta cont